In [1]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace


/workspace


In [2]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [3]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


# TODO

1. ✅CG, MGで時間をExtra-Pから取得できることを確認
2. ✅時間の単位を「秒」にする
    * 列「Inclusive total msec」に一部存在している”：”で区切られた値の単位は、msec ではなく分と秒を区切る”：”
3. ✅相対コストの算出関数の実装
4. ✅Extra-P のDocker環境の構築
4. Extra-P による予測との組み合わせを行う
    * なにをどう組み合わせるのかがわかっていないので、それは確かめる
    * 元データ, Extra-P単体で予測したデータ, <何か> で結果を作成

# 予測結果に必要なもの

* ✅元データ（予測環境の生データ）
* ✅Extra-P単体で予測したデータ
* Extra-Pで各関数の実行時間を予測し、それにコール回数を掛けた値で予測したデータ
    * コール回数で実行時間を割る
    * Extra-Pでモデルを作成 <- コール回数で実行時間を割る
    * 👆を利用して予測
    * 本プログラムでモデルを作成 <- コール回数で実行時間を割る
    * 👆を利用して予測
    * 「コール回数 * 実行時間」を計算して予測

In [4]:
# 元データによる結果を取得する
# ✅1. Extra-Pに読み込ますデータの取得
# 2. 取得できた数式から予測


input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

str_ExtraPinputData :str = gen_ExtraPinputData(
    benchmarkName = "cg",
    conditions={
        "process":input_list_process,
        "na":cg_input_list_na,
        "nonzer":cg_input_list_nonzer,
        "niter":cg_input_list_niter,
        "shift":cg_input_list_shift
    }
)

filePath = "./extra-p_docker/share/input_cg.txt"
with open(filePath, mode="w") as f:
    f.write(str_ExtraPinputData)


/workspace/lib/lab_lib.py:8781: UserWarning: ./csv_files/cg_na75000_nonzer18_niter30_shift60_process4.csv is empty.
  warnings.warn(f"{filePath} is empty.")
/workspace/lib/lab_lib.py:8781: UserWarning: ./csv_files/cg_na75000_nonzer15_niter90_shift80_process8.csv is empty.
  warnings.warn(f"{filePath} is empty.")


In [5]:
%sx pwd
%sx which extrap
%ls
%sx extrap --text /workspace/extra-p_docker/share/input_cg.txt

Dockerfile*    csv_files/           lib/                usedNote.ipynb*
Pipfile*       docker-compose.yml*  root_jupyter/       usedNote2.ipynb*
Pipfile.lock*  docs/                testing_pri.ipynb*  usedNote3.ipynb*
README.md*     draft.ipynb          testing_pub.ipynb*  usedNote4.ipynb*
app/           extra-p_docker/      txt_files/          used_scripts/


['',
 'Loading file: |          | [00:00<?]',
 'Loading file:   0%|          | [00:00<00:12]',
 'Traceback (most recent call last):',
 '  File "/usr/local/bin/extrap", line 8, in <module>',
 '    sys.exit(main())',
 '  File "/usr/local/lib/python3.10/site-packages/extrap/extrap/extrapcmd.py", line 139, in main',
 '    experiment = read_text_file(arguments.path, pbar)',
 '  File "/usr/local/lib/python3.10/site-packages/extrap/fileio/text_file_reader.py", line 133, in read_text_file',
 '    raise FileFormatError("This input format supports a maximum of 4 parameters.")',
 'extrap.util.exceptions.FileFormatError: This input format supports a maximum of 4 parameters.']

In [6]:
train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48, 64]

test_lulesh_processes: list[int] = [512]
test_lulesh_iterations: list[int] = [256]
test_lulesh_sizes: list[int] = [128]

str_ExtraPinputData :str = gen_ExtraPinputData(
    benchmarkName="lulesh",
    conditions = {
        "process" : train_lulesh_processes,
        "iteration" : train_lulesh_iterations,
        "size" : train_lulesh_sizes
    },
    version=2
)

filePath = "./extra-p_docker/share/input_lulesh.txt"
with open(filePath, mode="w") as f:
    f.write(str_ExtraPinputData)


In [7]:
%sx extrap --text /workspace/extra-p_docker/share/input_lulesh.txt

['',
 'Loading file: |          | [00:00<?]',
 'Loading file: 100%|██████████| [00:00<00:00, Creating calltree]',
 'Loading file: 100%|██████████| [00:00<00:00, Validating experiment]',
 'Loading file: 100%|██████████| [00:00<00:00, Validating experiment]',
 '',
 'Generating models: |          | [00:00<?]',
 'Generating models: 100%|██████████| [00:00<00:00]',
 'Callpath: reg',
 '\tMetric: time',
 '\t\tMeasurement point: (8.00E+00,8.00E+00,1.60E+01) Mean: 6.48E+00 Median: 6.48E+00',
 '\t\tMeasurement point: (8.00E+00,8.00E+00,2.40E+01) Mean: 8.36E+00 Median: 8.36E+00',
 '\t\tMeasurement point: (8.00E+00,8.00E+00,3.20E+01) Mean: 6.55E+00 Median: 6.55E+00',
 '\t\tMeasurement point: (8.00E+00,8.00E+00,4.80E+01) Mean: 6.43E+00 Median: 6.43E+00',
 '\t\tMeasurement point: (8.00E+00,8.00E+00,6.40E+01) Mean: 8.96E+00 Median: 8.96E+00',
 '\t\tMeasurement point: (1.60E+01,8.00E+00,1.60E+01) Mean: 6.12E+00 Median: 6.12E+00',
 '\t\tMeasurement point: (1.60E+01,8.00E+00,2.40E+01) Mean: 8.59E+00 Med

上記のセルよりモデルは次式

$$ 2.5521930556315375 + 0.013737041505459383 * iteration + 3.41165137739477 * 10^7 * iteration * size^3 * \log_2 size $$

In [8]:
rawDF_lulesh :pd.DataFrame = return_rawDFinLULESH (
    processes=train_lulesh_processes,
    iterations=train_lulesh_iterations,
    sizes=train_lulesh_sizes,
    csvDirPath=csvDirPath
)

rawDF_lulesh

,Name,#Call,process,iteration,size
0,.TAU_application,1.000000,8,8,16
1,int_main(int_char_**),1.000000,8,8,16
2,MPI_Finalize(),1.000000,8,8,16
3,MPI_Init(),1.000000,8,8,16
4,void_CalcKinematicsForElems(Domain,8.000000,8,8,16
...,...,...,...,...,...
27,void_Domain::SetupSymmetryPlanes(Int_t),1.000000,343,128,64
28,void_ParseCommandLineOptions(int_char_**_Int_t...,1.000000,343,128,64
29,StrToInt,2.000000,343,128,64
30,MPI_Comm_size(),1.000000,343,128,64


In [9]:
return_rawDF_lulesh(
    list_process = test_lulesh_processes,
    list_iteration = test_lulesh_iterations,
    list_size = test_lulesh_sizes,
    csvDir=csvDirPath
)

,%Time,Exclusive,Inclusive,#Call,#Subrs,Name,process,iteration,size
0,100.0,0.00348,19:09.925,1.000000e+00,1.0,.TAU_application,512,256,128
1,100.0,13:13.799,19:09.925,1.000000e+00,2828.0,int_main(int_char_**),512,256,128
2,16.8,1:53.429,3:12.675,2.560000e+02,536871000.0,void_CalcKinematicsForElems(Domain,512,256,128
3,9.6,1:50.861,1:50.861,2.550000e+02,0.0,MPI_Allreduce(),512,256,128
4,6.9,1:19.552,1:19.552,5.389680e+08,0.0,Real_t_CalcElemVolume(const,512,256,128
5,2.9,"2,037","33,621",7.690000e+02,10503.8,void_CommSend(Domain,512,256,128
6,2.7,"31,334","31,334",7.690000e+02,0.0,MPI_Waitall(),512,256,128
7,1.1,"12,869","12,869",8.965800e+03,0.0,MPI_Wait(),512,256,128
8,0.7,"2,773","7,888",2.560000e+02,2790.0,void_CommSyncPosVel(Domain,512,256,128
9,0.6,660,"7,184",2.570000e+02,5344.8,void_CommSBN(Domain,512,256,128


In [10]:
%reset

Nothing done.


***

---

___

# 2022年4月17日～

次のような表を作成する

採用される割合 (MAPE の最大値 [%] ，MAPE の最小値 [%]) [%]

| ベンチマークプログラム名 | 線形モデル               | 対数モデル               | 反比例モデル              |
|--------------|---------------------|---------------------|---------------------|
| str          | float(float, float) | float(float, float) | float(float, float) |


目標となるのは一気にこのベンチマークプログラムを作成することだが、既存のライブラリ関数などを利用し、まずはベンチマークごとに作成可能にする。

メモ

## 実装予定

1. 行方向に最小値を検出
2. 最小値以外をNaNに変更
3. 列方向に最小値と最大値を検出

## 


***

---

___